Code to interactively extract data from the full slave voyages spreadsheet, written for HUM2059 in November 2025


In [ ]:
# Install dependencies quietly
!pip install -q pandas ipywidgets openpyxl

import pandas as pd
from ipywidgets import widgets, VBox, HBox, Button, Layout, Output
from IPython.display import display, clear_output
from google.colab import files
import io

# Step 1: Upload file
print("Upload your CSV or XLSX file:")
upload_output = Output()
with upload_output:
    uploaded = files.upload()

file_name = list(uploaded.keys())[0]
if file_name.endswith(".csv"):
    df = pd.read_csv(io.BytesIO(uploaded[file_name]))
else:
    df = pd.read_excel(io.BytesIO(uploaded[file_name]))

clear_output()  # hide upload details
print(f"Loaded '{file_name}' with {len(df.columns)} columns.")

# Step 2: Choose columns to KEEP
checkboxes = [widgets.Checkbox(value=True, description=col) for col in df.columns]
keep_button = Button(description="Confirm columns to KEEP", button_style="success")
keep_box = VBox(checkboxes + [keep_button])
display(keep_box)

selected_cols = []
merge_definitions = []  # store multiple merge instructions

def on_keep_clicked(b):
    global selected_cols
    selected_cols = [cb.description for cb in checkboxes if cb.value]
    clear_output()
    print(f"Columns selected to keep: {selected_cols}")
    merge_interface()

keep_button.on_click(on_keep_clicked)

# Step 3: Merge interface with repeat option
def merge_interface():
    merge_label = widgets.Label("Select columns to MERGE (you can repeat merges):")
    merge_select = widgets.SelectMultiple(options=selected_cols, description="Columns:")
    merge_name = widgets.Text(value="MergedColumn", description="New name:")
    merge_sep = widgets.Text(value=", ", description="Separator:")
    add_merge_btn = Button(description="Add Merge", button_style="info")
    done_btn = Button(description="Finish & Download", button_style="success")
    merge_list_output = Output()
    preview_output = Output()
    main_output = Output()

    def add_merge_action(b):
        if not merge_select.value:
            with merge_list_output:
                print("Please select at least one column to merge.")
            return
        merge_definitions.append({
            "cols": list(merge_select.value),
            "name": merge_name.value,
            "sep": merge_sep.value
        })
        with merge_list_output:
            clear_output()
            print("Added merge definitions:")
            for m in merge_definitions:
                print(f" - {m['name']}: {m['cols']} (sep='{m['sep']}')")

    def finish_action(b):
        with main_output:
            clear_output()
            result = df[selected_cols].copy()
            for m in merge_definitions:
                result[m["name"]] = df[m["cols"]].astype(str).agg(m["sep"].join, axis=1)
            csv_bytes = result.to_csv(index=False).encode("utf-8")
            fname = "selected_columns.csv"
            with open(fname, "wb") as f:
                f.write(csv_bytes)
            files.download(fname)
            print(f"File '{fname}' ready for download with all merges applied.")

    add_merge_btn.on_click(add_merge_action)
    done_btn.on_click(finish_action)

    ui = VBox([
        merge_label,
        merge_select,
        HBox([merge_name, merge_sep]),
        HBox([add_merge_btn, done_btn]),
        merge_list_output,
        main_output
    ])
    display(ui)


Columns selected to keep: ['Voyage ID', 'Year arrived with captives', 'Total disembarked (IMP)', 'Total embarked (IMP)', "Duration of captives' crossing (in days)", 'Voyage duration, homeport to disembarkation (in days)', 'Crew at first landing of captives', 'Crew at voyage outset', 'Standardized Tonnage (IMP)', 'Sterling cash price in Jamaica (IMP)', 'Percentage of captives who died during crossing (IMP)', 'Percent women', 'Outcome of voyage if ship captured', 'Flag of vessel (IMP)', "Region where vessel's voyage ended (IMP)", 'Rig of Vessel', "Place where vessel's voyage ended", 'Percent boys', 'Principal region of captive disembarkation (IMP)', "Place where vessel's voyage began (IMP)", 'Month first disembarkation of captives', 'Broad region of captive disembarkation (IMP)', 'Percent girls', 'Particular outcome', 'Principal place where captives were landed (IMP)', 'Percent children', 'Percent women.1', 'Month departed last place of landing', 'Outcome of voyage for investors', 'Broad

The code below is not meant for user execution but needed to clean the notebook before pushing it to GitHub. Ignore this code and focus on the cell above when you are reusing the code.

In [ ]:
# Clean notebook widget metadata
!pip install -q nbformat

import nbformat, os
from google.colab import drive, files

# Mount Google Drive
drive.mount("/content/drive")

# Check notebook name
notebook_name = input("Enter your notebook filename (e.g. my_notebook.ipynb): ").strip()
if not notebook_name:
    raise ValueError("Notebook name is required.")

# Build full path inside Drive
base_path = "/content/drive/MyDrive/Colab Notebooks"
path = os.path.join(base_path, notebook_name)
if not os.path.exists(path):
    raise FileNotFoundError(f"Notebook not found at {path}")

# Read, strip widget metadata, and write clean copy
with open(path) as f:
    nb = nbformat.read(f, as_version=4)

if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]

clean_path = os.path.join(base_path, f"clean_{notebook_name}")
with open(clean_path, "w") as f:
    nbformat.write(nb, f)

print(f"\n✅ Cleaned notebook created:\n{clean_path}\n")

Mounted at /content/drive
